In [2]:
import pandas as pd
import json
import os

# Json parsing

In [81]:
data = []
mydf = pd.DataFrame()

In [82]:
#현재 경로 확인
%pwd

'c:\\dev\\github\\analysis practice\\로그 데이터 분석'

In [83]:
#to parse json files
with open('./datas/tutorial_01/ds_tutorial_json.json') as f:
    for line in f:
        data.append(json.loads(line))

    # convert json to df
    for i in range(0,len(data)):
        df = pd.DataFrame.from_dict([data[i]])
        mydf =pd.concat([mydf,df])

    mydf.reset_index(drop=True, inplace = True)
        

In [84]:
mydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    120 non-null    object
 1   memno   120 non-null    object
 2   screen  120 non-null    object
 3   event   120 non-null    object
 4   ver     120 non-null    object
 5   area    120 non-null    object
 6   group   120 non-null    object
dtypes: object(7)
memory usage: 6.7+ KB


In [85]:
mydf['date']

0      1511947418
1      1511947418
2      1511947418
3      1511947418
4      1511947418
          ...    
115    1511688217
116    1511688217
117    1511688217
118    1511688217
119    1511688217
Name: date, Length: 120, dtype: object

In [88]:
#change date column type
mydf['date'] = pd.to_datetime(mydf['date'],unit='s').dt.date
mydf['date'] = mydf['date'].astype('datetime64[ns]')

# Python SQL 
- pandasql

In [17]:
%cd C:\dev\github\analysis practice\로그 데이터 분석\datas\

C:\dev\github\analysis practice\로그 데이터 분석\datas


In [27]:
#문서 이용행동 로그 csv load -> sampling
df = pd.read_csv('tutorial_01\doc_use_log.csv').sample(frac =0.01 , replace=False)
df

,actiontype,ismydoc,ext,sessionid,documentposition,datetime
93942,OPEN,False,TXT,36b4082a5e11e60f15cb927de0755427,OTHERAPP,2016.7.13
91087,CLOSE,False,HWP,ee73f250a15c88f2bc11f7efcde777f9,OTHERAPP,2016.7.7
271928,CLOSE,False,DOCX,6bc42eafd2c100de41ea2023d3213de1,OTHERAPP,2016.7.17
45906,CLOSE,True,XLS,a0e9c49ef62c69c9851008d01707c6a1,MYPOLARISDRIVE,2016.7.16
221520,OPEN,False,PDF,2cb1bf79638effe33eea91c4e4e5b611,OTHERAPP,2016.7.21
...,...,...,...,...,...,...
75495,CLOSE,False,PPTX,72afc200afdc5468c4066e1e89b68ee0,OTHERAPP,2016.7.24
61089,OPEN,False,PDF,b50ae4a2bd7f39d410ba49d34608c675,OTHERAPP,2016.7.6
145162,OPEN,True,HWP,408326c6eb30b6edfbbaa2b34829406f,MYPOLARISDRIVE,2016.7.13
174909,CLOSE,True,PDF,a7facac97a7340a7d7584092c36104c3,MYPOLARISDRIVE,2016.7.1


In [48]:
#sql 언어 활용을 위한 라이브러리, 쿼리 작성
from pandasql import *

q = "SELECT * FROM df  WHERE ext = 'PDF' AND ismydoc = '0' LIMIT 10 "
sqldf(q,locals())


,actiontype,ismydoc,ext,sessionid,documentposition,datetime
0,OPEN,0,PDF,2cb1bf79638effe33eea91c4e4e5b611,OTHERAPP,2016.7.21
1,CLOSE,0,PDF,41665024f554e114453bf4db000b9fd7,OTHERAPP,2016.7.13
2,CLOSE,0,PDF,3981e2a92925c6cc72058a21c0cb90a9,LOCALSTORAGE,2016.7.12
3,CLOSE,0,PDF,ae88164bc9050e85cdf84921ce2802f0,OTHERAPP,2016.7.18
4,OPEN,0,PDF,708e659c8e52d7c3f2fb397a4041666e,OTHERAPP,2016.7.12
5,CLOSE,0,PDF,a2dcb45e88a87d263f2d1de080668852,OTHERAPP,2016.7.6
6,CLOSE,0,PDF,463eff6795a7f19ebbe33357ea18860b,OTHERAPP,2016.7.1
7,OPEN,0,PDF,19c1b755395fb32c8658257b7b7bbd68,OTHERAPP,2016.7.16
8,CLOSE,0,PDF,f82db7acaa28900db8e651f1c4a80235,LOCALSTORAGE,2016.7.24
9,CLOSE,0,PDF,bf76159f286b65231e2d96921d774212,OTHERAPP,2016.7.25


In [56]:
#SQL group by 함수를 이용한 집계 처리 - 문서별 사용자 수
q = """
        select ext,
        count(ext) as count,
        count(distinct sessionid) as unq_sess
        from df
        group by ext
        order by count DESC
"""
sqldf(q,locals())

,ext,count,unq_sess
0,PDF,842,830
1,DOCX,571,564
2,XLSX,502,498
3,HWP,279,278
4,DOC,263,263
5,XLS,245,243
6,PPTX,145,145
7,TXT,92,92
8,PPT,55,54
9,PPSX,14,14


In [59]:
# ios 사용자 현황 csv load
ios = pd.read_csv('tutorial_01\ios.csv')
ios.head()

,idx,sessionid,flag
0,1331,b8a35b63d65e36fcbcf7284db362fa06,iOS
1,1335,ab9ea758f63a61463efa059c7183b976,iOS
2,1383,401e95152f65bcd9573bb1707967e82a,iOS
3,1385,7891c7016850ccca28b54b3f8f0c554e,iOS
4,1387,dd16321402b19a3f20261e6905e4e9bf,iOS


In [62]:
#join tabel
q = """
    select 
    A.*,
    B.flag
    from df A
    left join
    (select sessionid, flag
    from ios
    ) B
    on A.sessionid = B.sessionid
    where B.flag = 'iOS'
"""

sqldf(q,locals())

,actiontype,ismydoc,ext,sessionid,documentposition,datetime,flag
0,OPEN,1,HWP,865e379a9320bb9a9ea308ac52d9212e,OTHERAPP,2016.7.9,iOS
1,OPEN,0,XLSX,fc97a885d8b719c7b840f87d42b434c2,OTHERAPP,2016.7.8,iOS
2,CLOSE,0,DOCX,8a24adbab0d9d4ef22f7e6213381a223,OTHERAPP,2016.7.7,iOS


## Question 1 
1. df 테이블의 Action Type 값(항목)별 유니크한 세션수는?
2. 유니크 세션수 기준으로 내림차순 정리


In [74]:
df.head()

,actiontype,ismydoc,ext,sessionid,documentposition,datetime
93942,OPEN,False,TXT,36b4082a5e11e60f15cb927de0755427,OTHERAPP,2016.7.13
91087,CLOSE,False,HWP,ee73f250a15c88f2bc11f7efcde777f9,OTHERAPP,2016.7.7
271928,CLOSE,False,DOCX,6bc42eafd2c100de41ea2023d3213de1,OTHERAPP,2016.7.17
45906,CLOSE,True,XLS,a0e9c49ef62c69c9851008d01707c6a1,MYPOLARISDRIVE,2016.7.16
221520,OPEN,False,PDF,2cb1bf79638effe33eea91c4e4e5b611,OTHERAPP,2016.7.21


In [90]:
#answer 1
q = """
    select actiontype,
    count(distinct sessionid) as unq_sess_cnt
    from df
    group by actiontype
    order by unq_sess_cnt desc
    """

sqldf(q,locals())

,actiontype,unq_sess_cnt
0,OPEN,1502
1,CLOSE,1294
2,SAVE,98
3,RESET,70
4,SAVEAS,43
5,EXPORT,1


## Question 2
1. ismydoc이 1(True)인 경우에 한해, 날짜별 세션수의 유니크 빈도 구하기
2. 유니크 빈도가 가장 큰 top 5 날짜 확인하기

In [91]:
# answer 2
q = """
    select datetime,
    count(distinct sessionid) as unq_sess_cnt
    from df
    where ismydoc = '1'
    group by datetime
    order by unq_sess_cnt desc
    limit 5
    """

sqldf(q,locals())

,datetime,unq_sess_cnt
0,2016.7.14,52
1,2016.7.1,52
2,2016.7.26,50
3,2016.7.20,49
4,2016.7.19,49


## Question 3
1. 문서 포지션별로 자주 open되는 확장자(ext)를 확인하기
2. 카운트 기준 : unique sessionid
3. 그룹별, 세션카운트 기준 내림차순 정렬

In [96]:
# answer 3
q = """
    select documentposition,
    ext,
    count(distinct sessionid) as unq_sess_cnt
    from df
    where actiontype = 'OPEN'
    group by documentposition , ext
    order by documentposition, unq_sess_cnt desc
    """

sqldf(q,locals())

,documentposition,ext,unq_sess_cnt
0,LOCALSTORAGE,HWP,22
1,LOCALSTORAGE,XLSX,18
2,LOCALSTORAGE,PDF,18
3,LOCALSTORAGE,DOCX,10
4,LOCALSTORAGE,XLS,7
5,LOCALSTORAGE,PPTX,6
6,LOCALSTORAGE,DOC,6
7,LOCALSTORAGE,TXT,3
8,LOCALSTORAGE,PPT,2
9,MYPOLARISDRIVE,PDF,65
